Like in the parsing notebook, we're going to prepare data from three sources (molecular, morphological and fossils without data, used only for their age information) for use with the BEAST2 suite of tools for divergence dating using the Fossilized Birth-Death model. This time around, we're going to subsample the fossil data - including every fossil, especially when they all have missing data, can make it really hard for the MCMC to converge. 

In [1]:
import dendropy
import pandas as pd
from numpy import random

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

The first thing we do is load our the two libraries we'll be using: Pandas, a data-crunching library and Dendropy, a perennially excellent package for phylogenetic tree and dataset manipulation.

In the previous notebook, we parsed all the data into dataframes with extra info, such as higher taxonomy and ages. We're going to load that data now.

In [3]:
morphTable = pd.read_csv("../Data/Morph/morphTNRS.csv")
molTable = pd.read_csv("new_df.csv")
fossTable = pd.read_csv("../Data/Morph/FossilTNRS.csv")

In order to subsample the fossils, we're first going to break them down by subfamily. We will then subsample within each subfamily.

In [4]:
families = fossTable.groupby('subfamily')

The above defines some probability that a fossil will be included in the final set of fossils. I set this so that about 20% of the fossils would be included. This will still be a lot of fossils! Then, below, I put this into a dataframe so that I can merge my subset of fossils with my molecular and morphological data.

In [5]:
samp = pd.DataFrame(molTable, columns=['specimen', 'subfamily'])

NameError: name 'sample' is not defined

In [6]:
morphMerge = morphTable[['Specimen', 'SubFamily']]
morphMerge.columns = ['specimen','subfamily'] 
morphMerge['Fossil'] = 'No'
molMerge = molTable[['specimen', 'subfamily']]
molMerge.columns = ['specimen','subfamily']  
molMerge['Fossil'] = 'No'
samp['Fossil'] = 'Yes'
mega_df = pd.concat([samp,molMerge,morphMerge]) 
mega_df = mega_df.drop_duplicates('specimen')

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


NameError: name 'samp' is not defined

Now we load in DNA and character matrices.

In [2]:
molDat = dendropy.DnaCharacterMatrix.get_from_path("../Data/Mol/FINAL_666Trimmed.nex", schema="nexus",preserve_underscores=True)  


In [ ]:
sample = []
samp_prop = .9

for name, group in families:
    for ind in group.specimen:
        ind_val = random.random(1)
        if ind_val > samp_prop:
            sample.append([ind, name])
sample = pd.DataFrame(sample)
sample.columns = ['Specimen', 'subfamily']
sample['Fossil'] = 'Yes'


In [ ]:
morphDat = dendropy.StandardCharacterMatrix.get_from_path("../Data/Morph/KellerMatrix.nex", schema="nexus", preserve_underscores=True)


In [ ]:
for taxon in morphDat.taxon_namespace:
    if taxon not in molDat.taxon_namespace:
        molDat.taxon_namespace.add_taxon(str(taxon))
print(molDat.taxon_namespace)

In [ ]:
morphDat = dendropy.StandardCharacterMatrix.get_from_path("../Data/Morph/KellerMatrix.nex", schema="nexus", preserve_underscores=True, taxon_namespace=molDat)

And we create matrices of missing data for the fossils we're using for their age info - remeber that all taxa must be present in all data partitions in BEAST.

In [12]:
names=[]
names = sample.Specimen.tolist()

In [13]:
md_val = "?"*139
dict_of_dat = {}
print(len(names))
names = [name for name in names if name not in molDat.taxon_namespace]
print(len(names))
names_x = [name+'_X' for name in names]
for name in names_x:                                                                                             
	dict_of_dat[name] = md_val
dict_of_moldat = {}
md_val = "?"*7099
for name in names_x:                                                                                             
	dict_of_moldat[name] = md_val

81
81


We make sure that the names of the fossils are entered into the namespace.

In [14]:
fossDat = dendropy.StandardCharacterMatrix.from_dict(dict_of_dat,taxon_namespace=molDat.taxon_namespace) 
fossmolDat = dendropy.DnaCharacterMatrix.from_dict(dict_of_moldat, taxon_namespace=molDat.taxon_namespace)    

AttributeError: 'str' object has no attribute 'lower_cased_label'

And if there are any morphology taxa that don't have molecular data, or vice versa, we pad that out now as well.

In [ ]:
#fossDat.replace_sequences(morphDat)
morphDat.pack()
#fossmolDat.replace_sequences(molDat)
molDat.pack()

And we write the data out. Pack() fills the data with None values. These can be replaced by regex. Once you have done this, BEAST should happily read the data. 

In [ ]:
morphDat.write_to_path('samp_morphTest.nex', schema='nexus')
molDat.write_to_path('samp_molTest.nex', schema='nexus')

For the next bits, I'm going to show some parsing that will be useful if you want to use fossil tip ranges, such as in [this](http://sysbio.oxfordjournals.org/content/early/2016/07/27/sysbio.syw060.abstract) paper. In order to access the age information stored in my fossil data, I'm going to do merge between my fossil subsample, and my fossil database. This will allow me to pare down my fossil database to just the fossils I actually want to use.

In [ ]:
fossTNRS = pd.read_csv('../Data/Morph/FossilTNRS.csv')

In [ ]:
fossTNRS

If you look at any of the XML files for the paper I linked to in the previous text, you'll see taxon names and ages are stored together in an XML block. Below, we assemble this XML block for our data.

In [ ]:
namesdb = fossTNRS[fossTNRS['specimen'].isin(names)]
nl = namesdb['specimen'].tolist()
nm = namesdb['min_ma'].tolist()

new_names = [name+'_X='+str(time) for name, time in zip(nl, nm)]
new_names

Looking at the XMLs, you might also notice that we can sample the tip ages, if we have an age range for a fossil. Below, I make the part of the XML block that varies. 

In [ ]:
xml_taxon =  ['<taxon idref="'+name+'_X " spec="Taxon"/>' for name in names] 

In [ ]:
xml_taxon

In [ ]:
count = 1
nl = namesdb['specimen'].tolist()
nm = namesdb['min_ma'].tolist()
nma = namesdb['max_ma'].tolist()

for name, mi, ma in zip(nl, nm, nma):
    print('<samplingDates id="samplingDate%d" spec="beast.evolution.tree.SamplingDate" taxon="@'  % count +name+'_X" lower="'+str(mi)+'" upper="'+str(ma)+'"/>')
    count=count+1

    
    #sample_taxon =  ['<samplingDates id="samplingDate%d" spec="beast.evolution.tree.SamplingDate" taxon="@'+name+'" lower="'+str(mi)+'" upper="'+str(ma)+'"/>' for name,mi,ma in zip(names, merged.min_ma, merged.max_ma) %count count=count+1]

To subsample tips involves some amount of manual assembly of XML. You can plug and chug this by copying the XML structure from the XML files associated with the paper. Below, I take the dataframe of all the taxa in our analysis and use it to constrain monophyloetic groups at the subfamily level. You can also create these in BEAUTi, but I really hate clicking things in a GUI, so i'm doing it here.

In [ ]:
x_names = [name+'_X' for name in nl]

namesdb['specimen'] = x_names
total = namesdb.append([molMerge, morphMerge])


In [ ]:
total

In [ ]:
families = total.groupby('subfamily')
for name, group in families:
    print('<distribution id="'+name+'.prior" spec="beast.math.distributions.MRCAPrior" monophyletic="true" tree="@Tree.t:samp_morphTest">')
    print('<taxonset id="'+name+'" spec="TaxonSet">')
    for ind in set(group.specimen):
        print('<taxon id="'+ind+'" spec="Taxon"/>')
    print('</taxonset>'+'\n'+'</distribution>')  
            
#    fname = name
#   group.specimen.to_csv(fname, index=False)

I'm interested in looking at the posterior distribution of ages for all the ant clades, so I want to add those to the operators so that information gets written to the log. The below formats the ops block.

In [ ]:
for name, group in families:
    print(' <log idref="'+name+'.prior"/>')

As I'm wrapping up this post, I am realizing I ought to do a schematic of the BEAST XML file to make it easier to understand the chunks of it if you're not an XML whiz.

In [ ]:
for name, group in families:
    for ind in group.specimen:
        if ind in names or molDat.taxon_namespace:
            print('yes')
        else:
            print('no')

In [ ]:
for x in molDat.taxon_namespace:
    print(x)